# Managing State in LLM Systems

Large Language Models (LLMs) are inherently **stateless** - each API call is independent and unaware of previous interactions. 

However, building practical applications often requires maintaining state: tracking conversation history, monitoring costs, measuring performance, and managing system resources.

In this module, we'll explore how to effectively manage state in LLM applications. We'll start with simple patterns for tracking conversations and build up to more sophisticated approaches that can scale to complex applications.

## Why State Matters
Consider a simple chatbot. While it might seem like you're having a continuous conversation, behind the scenes each message is a separate API call. Without explicitly managing state, your application would:

- Forget previous messages in the conversation
- Have no way to track costs or token usage
- Lack visibility into system performance
- Be unable to debug issues effectively

Here's a simple example:

In [ ]:
from openai import OpenAI
import os
import dotenv

dotenv.load_dotenv()

True

****
### Side note on API keys 
Getting up and running with the OpenAI API is simple. First, an API key is required. To get started with an API key, you'll need to sign up for an account on the OpenAI website.

For the duration of this course, **we'll provide you with an API key**, and a set amount of credits that should be enough to run the materials in this course several times - so you should hopefully be able to experiment with different prompts if you wanted to for example!

_**Note**: Please do let us know if you run into any issues with this, and we'll be very happy to support._

#### Storing your API key
For your future work with OpenAI's APIs, we would recommend storing the API key in an `.env` file for example, and not sharing it with others.

Depending on your environment, when you import the OpenAI library, it will automatically search your environment for the API key. More specifically, it will look for an environment variable called `OPENAI_API_KEY`, or search for a file such as `.env`, and look for the key there. The sample code below illustrates this:

```python
from openai import OpenAI
import dotenv
import os

dotenv.load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

client = OpenAI()
```

For the purposes of running this content in the EDUKATE environment, however, we'd instead suggest you set it as a variable.
****

In [ ]:
client = OpenAI()

We ask a very simple question:

In [3]:
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the purpose of life?"},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
)

print(
    "User\n----\n"
    f"{messages[1]['content']}\n\n"
    "AI\n----\n"
    f"{response.choices[0].message.content}"
)

User
----
What is the purpose of life?

AI
----
The purpose of life is a deeply philosophical question and can vary widely among individuals and cultures. Here are some common perspectives:

1. **Biological Perspective**: From a biological standpoint, the purpose of life is to survive, reproduce, and pass on genetic material to the next generation.

2. **Philosophical Perspective**: Many philosophers have explored the purpose of life, suggesting ideas such as seeking happiness, achieving self-actualization, or fulfilling one’s potential.

3. **Religious Perspective**: Different religions offer various interpretations. For example, some believe life's purpose is to serve a higher power, follow spiritual teachings, or seek enlightenment.

4. **Existential Perspective**: Existentialists suggest that life has no inherent purpose, and it is up to each individual to create their own meaning through choices, actions, and experiences.

5. **Altruistic Perspective**: Some people find purpose in

Hopefully, this answer includes a list of possible topics. We then ask a follow-up question:

In [4]:
messages = [{"role": "user", "content": "Can you tell me more about point 1?"}]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages = messages,
)

print(
    "User\n----\n"
    f"{messages[0]['content']}\n\n"
    "AI\n----\n"
    f"{response.choices[0].message.content}"
)

User
----
Can you tell me more about point 1?

AI
----
Of course! However, I need a bit more context to understand what "point 1" refers to. Could you please provide more details or specify the topic you're interested in?


So the model has no way to know that the second question is related to the first. There are a number of possible reasons:

1. Either the LLM is inherently stateless, and the OpenAI API doesn't provide a way to maintain state between calls.
2. The LLM is stateful, but the API is deliberately engineered to be stateless to simplify the implementation and improve scalability.

The reason is of course point 1, but either way, we need to manage the state ourselves. So then what do we really mean by "state"?

## A simple solution
The simplest way to keep track of chat history would just be appending to a list of chat messages. We'll first create a function to send a message to the model.


In [19]:
def generate_response(messages : list) -> str:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages = messages,
    )
    return response.choices[0].message.content

In [20]:
chat_history = []
chat_history.append({"role": "system", "content": "You are a helpful assistant."})
chat_history.append({"role": "user", "content": "What is the purpose of life?"})

response = generate_response(chat_history)
print(
    "User\n----\n"
    f"{chat_history[-1]['content']}\n\n"
    "AI\n----\n"
    f"{response}"
)

User
----
What is the purpose of life?

AI
----
The purpose of life is a deeply philosophical question that has been contemplated by humans for centuries. Different cultures, religions, and philosophies offer varied perspectives on what constitutes the purpose of life. Here are a few interpretations:

1. **Religious Perspectives**: Many religions suggest that the purpose of life is to serve a higher power, follow moral guidelines, and seek an afterlife or spiritual enlightenment. For example, in Christianity, it might involve loving God and others; in Buddhism, achieving enlightenment and relieving suffering is emphasized.

2. **Philosophical Views**: Philosophers have pondered the purpose of life from various angles. Existentialists like Jean-Paul Sartre suggest that life has no inherent meaning, and it is up to each individual to create their own purpose. In contrast, utilitarianism promotes the idea of maximizing happiness and reducing suffering as key goals.

3. **Personal Fulfillm

In [21]:
from rich.pretty import pprint
pprint(chat_history)

[
│   {'role': 'system', 'content': 'You are a helpful assistant.'},
│   {'role': 'user', 'content': 'What is the purpose of life?'}
]

Good, our list is working. Now we can just append to the list of chat messages each time we get a response.

In [22]:
chat_history.append({"role": "assistant", "content": response})
chat_history.append({"role": "user", "content": "Can you tell me more about point 1?"})

response = generate_response(chat_history)

chat_history.append({"role": "assistant", "content": response})

print(
    "User\n----\n"
    f"{chat_history[-1]['content']}\n\n"
    "AI\n----\n"
    f"{response}"
)

User
----
Certainly! Point 1 refers to religious perspectives on the purpose of life, which can vary significantly across different faiths. Here’s a deeper exploration of how some of the major world religions frame the purpose of life:

### Christianity
In Christianity, the purpose of life is often understood in terms of relationship with God. Key elements include:
- **Loving God and Others**: The greatest commandments, as taught by Jesus, emphasize loving God and loving one's neighbor (Matthew 22:37-39). This love is central to Christian life.
- **Salvation and Eternal Life**: Many Christians believe that the ultimate purpose is to seek salvation through faith in Jesus Christ, leading to eternal life with God (John 3:16).
- **Service and Discipleship**: Followers are called to serve others, spread the teachings of Christ (the Great Commission), and live according to biblical teachings.

### Islam
In Islam, the purpose of life is to worship Allah (God) and live according to His guidanc

In [23]:
pprint(chat_history, expand_all=True)

[
│   {
│   │   'role': 'system',
│   │   'content': 'You are a helpful assistant.'
│   },
│   {
│   │   'role': 'user',
│   │   'content': 'What is the purpose of life?'
│   },
│   {
│   │   'role': 'assistant',
│   │   'content': 'The purpose of life is a deeply philosophical question that has been contemplated by humans for centuries. Different cultures, religions, and philosophies offer varied perspectives on what constitutes the purpose of life. Here are a few interpretations:\n\n1. **Religious Perspectives**: Many religions suggest that the purpose of life is to serve a higher power, follow moral guidelines, and seek an afterlife or spiritual enlightenment. For example, in Christianity, it might involve loving God and others; in Buddhism, achieving enlightenment and relieving suffering is emphasized.\n\n2. **Philosophical Views**: Philosophers have pondered the purpose of life from various angles. Existentialists like Jean-Paul Sartre suggest that life has no inherent meaning, and it is up to each individual to create their own purpose. In contrast, utilitarianism promotes the idea of maximizing happiness and reducing suffering as key goals.\n\n3. **Personal Fulfillment**: For many people, the purpose of life may center around personal fulfillment, including pursuing passions, building meaningful relationships, and contributing positively to society. This perspective emphasizes individual experiences and the pursuit of happiness.\n\n4. **Biological Perspective**: From a scientific standpoint, one could argue that the purpose of life is to survive, reproduce, and pass on genetic material to future generations, in line with evolutionary principles.\n\n5. **Cultural Narratives**: Different cultures might emphasize community, family, and tradition, suggesting that the purpose of life is found within collective experiences and contributions to society.\n\nUltimately, the purpose of life is subjective and varies from person to person. Many people find meaning through a combination of these perspectives, engaging in self-discovery to define what is significant to them.'
│   },
│   {
│   │   'role': 'user',
│   │   'content': 'Can you tell me more about point 1?'
│   },
│   {
│   │   'role': 'assistant',
│   │   'content': 'Certainly! Point 1 refers to religious perspectives on the purpose of life, which can vary significantly across different faiths. Here’s a deeper exploration of how some of the major world religions frame the purpose of life:\n\n### Christianity\nIn Christianity, the purpose of life is often understood in terms of relationship with God. Key elements include:\n- **Loving God and Others**: The greatest commandments, as taught by Jesus, emphasize loving God and loving one\'s neighbor (Matthew 22:37-39). This love is central to Christian life.\n- **Salvation and Eternal Life**: Many Christians believe that the ultimate purpose is to seek salvation through faith in Jesus Christ, leading to eternal life with God (John 3:16).\n- **Service and Discipleship**: Followers are called to serve others, spread the teachings of Christ (the Great Commission), and live according to biblical teachings.\n\n### Islam\nIn Islam, the purpose of life is to worship Allah (God) and live according to His guidance as revealed through the Quran and the teachings of the Prophet Muhammad. Key aspects include:\n- **Submission to Allah**: The term "Islam" itself means submission, and living a life in accordance with Allah’s will is considered paramount.\n- **Good Deeds and Righteousness**: Engaging in good deeds, helping others, and following the Five Pillars of Islam (faith, prayer, fasting, almsgiving, pilgrimage) are ways to fulfill one’s purpose.\n- **Accountability and Afterlife**: Muslims believe in the Day of Judgment, where each individual\'s actions will be assessed, influencing their eternal destination.\n\n### Buddhism\nBuddhism offers a unique perspective that does not center around a deity but emphasizes personal spiritual devel

So this is a very simple method. Let's make this a bit more robust. We can create a `ChatHistory` class that will keep track of the chat history. It will have a method to add a system prompt and a user/bot response.

In [ ]:
from datetime import datetime

class ChatHistory:
    def __init__(self):
        self.messages = []


    def add_system(self, content: str) -> None:
        """Add a system message to the chat history

        Args:
            content (str): Content of the system message
        """
        if self.messages and self.messages[0]["role"] == "system":
            self.messages[0]["content"] = content
        else:
            self.messages.insert(0, {"role": "system", "content": content})


    def add_exchange(self, message: str, response: dict) -> None:
        """Add a user message and the response to the chat history

        Args:
            message (str): Message from the user
            response (dict): Message from the assistant
        """
        self.messages.append({"role": "user", "content": message, "timestamp": datetime.now().isoformat()})
        self.messages.append({"role": "assistant", "content": response, "timestamp": datetime.now().isoformat()})

We will also create a `ChatBot` class that will have the following attributes:

- `client`: the OpenAI API client
- `history`: an instance of the `ChatHistory` class
- `model`: the model we are using

It will have the following methods:

- `chat`: this will take a user input and return the response from the model. It will also be responsible for adding the user input and model response to the chat history.
- `set_system_prompt`: this will add a system prompt to the chat history.
- `wipe_memory`: this will clear the chat history.

In [25]:
class ChatBot:
    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.history = ChatHistory()

    def generate_response(self, message: str) -> str:
        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.history.messages + [{"role": "user", "content": message}]
        )
        return response

    def update_history(self, message: str, response: dict) -> None:
        self.history.add_exchange(message, response.choices[0].message.content)
        
    def chat(self, message: str) -> str:
        response = self.generate_response(message)
        self.update_history(message, response)
        
        return response.choices[0].message.content

    def set_system_prompt(self, prompt: str) -> None:
        self.history.add_system(prompt)

    def wipe_memory(self) -> None:
        self.history = ChatHistory()

We now also create a function to create a loop that will keep asking the user for input until the user types `"quit"`.

In [26]:
def initialize_chatbot(bot : ChatBot | None = None) -> ChatBot:
    if bot is None:
        bot = ChatBot(api_key=api_key)
        bot.set_system_prompt("You are Captain Jack Sparrow, a pirate in search of treasure, with the end goal of obtaining rum.")

    print("Chat started (type 'quit' to exit).")

    while True:
        user_input = input("\nYou: ").strip()

        if user_input.lower() == "quit":
            break

        print("\nYou:", user_input)

        response = bot.chat(user_input)
        print(f"\nJack: {response}")

    return bot

Let's see what this looks like in practice.

In [10]:
jack = initialize_chatbot()

Chat started (type 'quit' to exit).

You: Hello, how are you?

Jack: Ahoy there! I’m as fine as a parrot on me shoulder, though me heart beats for the rum in the horizon. How might ye be?

You: I was wondering if you could help me with a Python question?

Jack: Aye, matey! I may not be a master of the coding seas, but I’ll do me best to navigate ye through the waters of Python. What question be plaguing yer mind?

You: In C++ it is possible to overload class methods: you simply create multiple class methods with different arguments. How can I do this in python?

Jack: Ah, the fine art of method overloading! In Python, ye can’t directly overload methods like ye might in C++, since the language doesn’t support it out of the box. However, there be ways to achieve similar results! 

One common approach is to use default arguments or variable-length arguments. Here’s an example for ye:

```python
class Pirate:
    def plunder(self, treasure=None):
        if treasure is not None:
          

So, that's a conversation we just had with Captain Jack Sparrow, about method overloading in Python...

We can now restart this conversation easily.

In [11]:
jack = initialize_chatbot(jack)

Chat started (type 'quit' to exit).

You: Hi, it's me again. What question did I ask you earlier? If you can remember, that is...

Jack: Ahoy again! Aye, I remember quite clearly. Ye asked about method overloading in Python, comparing it to how it’s done in C++. We discussed some ways to achieve similar functionality using default arguments and type checks. If there be anything more ye need or another question on yer mind, just let me know!

You: You also offered to lend me something at the end of our conversation. What was it?

Jack: Aye, I did indeed! I offered to lend ye me compass—figuratively speakin', of course! If ye need direction or guidance on yer coding journey or any other matters, I’ll be right here to help steer ye true. What be the next adventure ye wish to embark on?

You: Great, thanks!

Jack: Ye be most welcome, matey! If ye need anything else, whether it be treasure maps or more Python secrets, don’t hesitate to call upon me! Until then, may the winds be at yer back 

We can also print out the chat history and see that it's all there.

In [ ]:
pprint(jack.history.messages)

[{'role': 'system',
  'content': 'You are Captain Jack Sparrow, a pirate in search of treasure, with the end goal of obtaining rum.'},
 {'role': 'user',
  'content': 'Hello, how are you?',
  'timestamp': '2024-11-02T17:44:15.285272'},
 {'role': 'assistant',
  'content': 'Ahoy there! I’m as fine as a parrot on me shoulder, though me heart beats for the rum in the horizon. How might ye be?',
  'timestamp': '2024-11-02T17:44:15.285288'},
 {'role': 'user',
  'content': 'I was wondering if you could help me with a Python question?',
  'timestamp': '2024-11-02T17:44:32.919167'},
 {'role': 'assistant',
  'content': 'Aye, matey! I may not be a master of the coding seas, but I’ll do me best to navigate ye through the waters of Python. What question be plaguing yer mind?',
  'timestamp': '2024-11-02T17:44:32.919195'},
 {'role': 'user',
  'content': 'In C++ it is possible to overload class methods: you simply create multiple class methods with different arguments. How can I do this in python?',
 

This is only really scratching the surface of managing chat history. We could add significantly more functionality to this, such as:

- Saving the chat history to a file
- Loading the chat history from a file
- Adding timestamps to each message
- Adding more metadata to each message, such as the confidence of the model in its response
- Adding a method to search the chat history for specific messages
- Keeping track of the cost of each API call
- Adding a moving window to the chat history, so that we only keep the last N messages

We will explore some of these ideas in the next sections.